In [1]:
%pwd

'd:\\medical_chatbot_project\\Medical_chatbot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\medical_chatbot_project\\Medical_chatbot'

# Loading the dataset

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
#Extracting data from pdf
def load_pdf_data(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

In [10]:
extracted_data = load_pdf_data(data ="data/")

In [12]:
extracted_data[:10]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 2, 'page_label': '3'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 

In [13]:
#Splitting the extracted data into the chunks.
def split_data(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    chunks = splitter.split_documents(extracted_data)
    return chunks

In [15]:
chunks = split_data(extracted_data)
print(len(chunks))

5860


In [17]:
chunks[:10]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 2, 'page_label': '3'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 3, 'page_label': '4'}, page_content='STAFF

In [18]:
#Embedding the chunks 
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
#downloading the Hf embeddings 
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [21]:
embeddings = download_embeddings()

d:\medical_chatbot_project\Medical_chatbot\chatbot_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [23]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalchatbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [45]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [25]:
#creating embeddings for chunks and storing it in pinecone db
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    index_name = index_name,
    embedding= embeddings   
)

In [27]:
# Get the vectordb
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
docsearch

In [29]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
retrieved_docs = retriever.invoke("What is fever?")
retrieved_docs

[Document(id='0f476232-dee1-4f1a-8c47-d9bb3f34fed5', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 69.0, 'page_label': '70', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='fever in children. This disease is most often caused by\ntypes 3 and 7. Symptoms, which appear suddenly and\nusually disappear in less than a week, include:\n• inflammation of the lining of the eyelid (conjunctivitis)\n•f e v e r\n• sore throat (pharyngitis)\n• runny nose\n• inflammation of lymph glands in the neck (cervical\nadenitis)\nGALE ENCYCLOPEDIA OF MEDICINE 256\nAdenovirus infections\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 56'),
 Document(id='08e3d6d7-8b07-43cc-9abe-5e2139874488', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 60.0, 'page_label': '61', 'producer': 'PDFlib+PDI 5.0.0 (Su

In [ ]:
import os
from langchain_groq import ChatGroq

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

    llm = ChatGroq(model_name="llama-3.3-70b-versatile")  # or another available model
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015C62B89AC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015C62B8EA90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [78]:
#creating prompt
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [79]:
#Creating QA and RAG chain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [80]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bone and soft tissue. It occurs when this abnormality happens after bone growth stops. Gigantism, on the other hand, is the same condition, but it occurs when the abnormality happens before bone growth stops, resulting in unusual height. Both conditions are relatively rare, affecting approximately 50 out of every one million people. They can cause a variety of disturbances throughout the body, with symptoms often developing gradually.


In [81]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't know what "stats" refers to in this context. The provided text appears to discuss medical topics, including blood counts and athletic heart syndrome, but does not mention "stats" explicitly. If you have more information or context, I may be able to help further.
